In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ross_train=pd.read_csv('Rossmann Training Data.csv',dtype={'StateHoliday':str})
ross_store=pd.read_csv('Rossmann Store Data.csv',dtype={'StoreType':str})
ross_test=pd.read_csv('Rossmann Test Data.csv')



In [3]:
#No.of Unique stores and Dates in the Training Dataset
dates=ross_train.drop_duplicates(subset=['Date'])['Date'].count()
stores=ross_train.drop_duplicates(subset=['Store'])['Store'].count()
print("There are "+str(dates)+" unique dates in the training dataset")
print("There are "+str(stores)+" unique stores in the training dataset")


There are 942 unique dates in the training dataset
There are 1115 unique stores in the training dataset


In [4]:
# null values in the store dataset
ross_store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [5]:
ross_store=ross_store.fillna(0)
ross_store['Promo2SinceYear']=ross_store['Promo2SinceYear'].astype(int)
ross_store['Promo2SinceWeek']=ross_store['Promo2SinceWeek'].astype(int)
#Merging Training dataset with store information and changing object type of'Date' to datetime stamp
main_data=ross_train.merge(ross_store,on='Store',how='left')
main_data['Date']=pd.to_datetime(main_data.Date)
main_data.dtypes
#Adding additional columns-Year and month of sale and a concatenation of store type and assortment to aid analysis
year=pd.DataFrame(main_data.Date.dt.year)
year.columns=['Year of Sale']
month=pd.DataFrame(main_data.Date.dt.month)
month.columns=['Month of Sale']
concat=pd.concat([year,month],axis=1)
main_data=pd.concat([main_data,concat],axis=1)
main_data['StoreType+Assortment']=main_data['StoreType']+main_data['Assortment']
main_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year of Sale,Month of Sale,StoreType+Assortment
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0,0,0,2015,7,ca
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,570.0,11.0,2007.0,1,13,2010,"Jan,Apr,Jul,Oct",2015,7,aa
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,14130.0,12.0,2006.0,1,14,2011,"Jan,Apr,Jul,Oct",2015,7,aa
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,620.0,9.0,2009.0,0,0,0,0,2015,7,cc
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,29910.0,4.0,2015.0,0,0,0,0,2015,7,aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,...,1900.0,6.0,2014.0,1,31,2013,"Jan,Apr,Jul,Oct",2013,1,aa
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,...,1880.0,4.0,2006.0,0,0,0,0,2013,1,cc
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,...,9260.0,0.0,0.0,0,0,0,0,2013,1,ac
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,...,870.0,0.0,0.0,0,0,0,0,2013,1,ac


In [6]:
#Completeness Check #1
#There are 181 stores out of the 1115 stores that don't have data for all the 942 days in the dataset. 
z=pd.DataFrame(main_data.drop_duplicates(subset=['Store'])['Store'])
just_stores=z['Store'].tolist()
df=[]
for s in just_stores:
    x=main_data[main_data['Store']==s]['Store'].count()
    df.append(x)
y=pd.DataFrame(df,columns=['No. of Dates'])
store_data=pd.merge(z,y,left_index=True,right_index=True)
incomplete=store_data[store_data['No. of Dates']!=942]
w=incomplete['Store'].count()
inc_stores=incomplete['Store'].tolist()
impact=main_data[main_data['Store'].isin(inc_stores)]
b=impact['Store'].count()
a=main_data['Store'].count()
prop=round(100*b/a,2)
print('There are '+str(w)+' stores for which there is data for lesser than 942 days')
print('This impacts '+str(b)+' rows in the training dataset')
print('This is '+str(prop)+'% of the Dataset')

There are 181 stores for which there is data for lesser than 942 days
This impacts 137381 rows in the training dataset
This is 13.51% of the Dataset


In [7]:
year1_data=main_data[main_data['Year of Sale']==2013]
year2_data=main_data[main_data['Year of Sale']==2014]
year3_data=main_data[main_data['Year of Sale']==2015]
df1=[]
df2=[]
df3=[]
for st in inc_stores:
    eg1=year1_data[year1_data['Store']==st]['Date'].count()
    df1.append(eg1)
    eg2=year2_data[year2_data['Store']==st]['Date'].count()
    df2.append(eg2)
    eg3=year3_data[year3_data['Store']==st]['Date'].count()
    df3.append(eg3)
df0=pd.DataFrame(inc_stores)
df0.columns=['Store']
df1=pd.DataFrame(df1)
df1.columns=['No. of Dates in 2013']
df2=pd.DataFrame(df2)
df2.columns=['No. of Dates in 2014']
df3=pd.DataFrame(df3)
df3.columns=['No. of Dates in 2015']
incomplete_store_list=pd.concat([df0,df1],axis=1)
incomplete_store_list=pd.concat([incomplete_store_list,df2],axis=1)
incomplete_store_list=pd.concat([incomplete_store_list,df3],axis=1)
print('Total No. of Dates in 2013')
print(year1_data.drop_duplicates(subset=['Date'])['Store'].count())
print('Total No. of Dates in 2014')
print(year2_data.drop_duplicates(subset=['Date'])['Store'].count())
print('Total No. of Dates in 2015')
print(year3_data.drop_duplicates(subset=['Date'])['Store'].count())
print(incomplete_store_list)
print('Noted that some stores have 181 dates instead of 365 in 2014.Also noted that these stores have data in 2015(these were not shut down)')

Total No. of Dates in 2013
365
Total No. of Dates in 2014
365
Total No. of Dates in 2015
212
     Store  No. of Dates in 2013  No. of Dates in 2014  No. of Dates in 2015
0       13                   365                   181                   212
1       20                   365                   181                   212
2       22                   365                   181                   212
3       32                   365                   181                   212
4       36                   365                   181                   212
..     ...                   ...                   ...                   ...
176   1094                   365                   181                   212
177   1102                   365                   181                   212
178   1104                   365                   181                   212
179   1107                   365                   181                   212
180   1109                   365                   181      

In [8]:
#Just one store-Store #988 is short of one date in 2013
print(incomplete_store_list.drop_duplicates(subset='No. of Dates in 2013')['No. of Dates in 2013'].count())
print(incomplete_store_list.drop_duplicates(subset='No. of Dates in 2014')['No. of Dates in 2014'].count())
print(incomplete_store_list.drop_duplicates(subset='No. of Dates in 2015')['No. of Dates in 2015'].count())
print(incomplete_store_list[incomplete_store_list['No. of Dates in 2013']!=365])
print(incomplete_store_list[incomplete_store_list['No. of Dates in 2014']!=181])
print(incomplete_store_list[incomplete_store_list['No. of Dates in 2015']!=212])

2
2
1
     Store  No. of Dates in 2013  No. of Dates in 2014  No. of Dates in 2015
160    988                   364                   365                   212
     Store  No. of Dates in 2013  No. of Dates in 2014  No. of Dates in 2015
160    988                   364                   365                   212
Empty DataFrame
Columns: [Store, No. of Dates in 2013, No. of Dates in 2014, No. of Dates in 2015]
Index: []


In [9]:
#We note that for all the stores with incomplete information in 2014, there is no information from july 2014
ap=incomplete_store_list['Store'].tolist()
del ap[160]
impact1=main_data[main_data['Store'].isin(ap)]
impact1=impact1[impact1['Year of Sale']==2014]
pd.DataFrame(impact1.describe()['Month of Sale'])


,Month of Sale
count,32580.000000
mean,3.508287
std,1.709987
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,6.000000


In [10]:
#As discussed,we are removing these 180 stores from further analysis-Main_data contains the dataset with which we will do our analysis
main_data=main_data[~main_data.Store.isin(ap)]
main_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year of Sale,Month of Sale,StoreType+Assortment
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0,0,0,2015,7,ca
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,570.0,11.0,2007.0,1,13,2010,"Jan,Apr,Jul,Oct",2015,7,aa
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,14130.0,12.0,2006.0,1,14,2011,"Jan,Apr,Jul,Oct",2015,7,aa
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,620.0,9.0,2009.0,0,0,0,0,2015,7,cc
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,29910.0,4.0,2015.0,0,0,0,0,2015,7,aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,...,1900.0,6.0,2014.0,1,31,2013,"Jan,Apr,Jul,Oct",2013,1,aa
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,...,1880.0,4.0,2006.0,0,0,0,0,2013,1,cc
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,...,9260.0,0.0,0.0,0,0,0,0,2013,1,ac
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,...,870.0,0.0,0.0,0,0,0,0,2013,1,ac


In [11]:
#Consolidation of data by store type to aid analysis
store_list=main_data.drop_duplicates(subset=['Store'])['Store'].tolist()
year1_data=main_data[main_data['Year of Sale']==2013]
year2_data=main_data[main_data['Year of Sale']==2014]
year3_data=main_data[main_data['Year of Sale']==2015]
sales_13=[]
sales_14=[]
sales_15=[]
customers_13=[]
customers_14=[]
customers_15=[]
for n in store_list:
    data1=year1_data[year1_data['Store']==n]['Sales'].sum()
    sales_13.append(data1)
    data2=year2_data[year2_data['Store']==n]['Sales'].sum()
    sales_14.append(data2)
    data3=year3_data[year3_data['Store']==n]['Sales'].sum()
    sales_15.append(data3)
    data4=year1_data[year1_data['Store']==n]['Customers'].sum()
    customers_13.append(data4)
    data5=year2_data[year2_data['Store']==n]['Customers'].sum()
    customers_14.append(data5)
    data6=year3_data[year3_data['Store']==n]['Customers'].sum()
    customers_15.append(data6)
sales_13=pd.DataFrame(sales_13)
sales_14=pd.DataFrame(sales_14)
sales_15=pd.DataFrame(sales_15)
customers_13=pd.DataFrame(customers_13)
customers_14=pd.DataFrame(customers_14)
customers_15=pd.DataFrame(customers_15)
store_data=pd.DataFrame(store_list).reset_index(drop=True)
store_data=pd.concat([store_data,sales_13],axis=1)
store_data=pd.concat([store_data,sales_14],axis=1)
store_data=pd.concat([store_data,sales_15],axis=1)
store_data=pd.concat([store_data,customers_13],axis=1)
store_data=pd.concat([store_data,customers_14],axis=1)
store_data=pd.concat([store_data,customers_15],axis=1)
store_data.columns=['Store','2013-Sales','2014-Sales','2015-Sales','2013-Customers','2014-Customers','2015-Customers']
store_data=store_data.merge(ross_store,on='Store',how='left')
store_data['StoreType+Assortment']=store_data['StoreType']+store_data['Assortment']
store_data['Total Sales']=store_data['2013-Sales']+store_data['2014-Sales']+store_data['2015-Sales']
store_data['Total Customers']=store_data['2013-Customers']+store_data['2014-Customers']+store_data['2015-Customers']
store_data

,Store,2013-Sales,2014-Sales,2015-Sales,2013-Customers,2014-Customers,2015-Customers,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType+Assortment,Total Sales,Total Customers
0,1,1491140,1433408,792306,178804,168764,92955,c,a,1270.0,9.0,2008.0,0,0,0,0,ca,3716854,440523
1,2,1488164,1516432,879262,177796,178057,102002,a,a,570.0,11.0,2007.0,1,13,2010,"Jan,Apr,Jul,Oct",aa,3883858,457855
2,3,2128265,2072949,1207047,232946,224897,126467,a,a,14130.0,12.0,2006.0,1,14,2011,"Jan,Apr,Jul,Oct",aa,5408261,584310
3,4,2852667,2971989,1731851,399175,405758,231321,c,c,620.0,9.0,2009.0,0,0,0,0,cc,7556507,1036254
4,5,1420228,1406465,816125,163857,161960,92771,a,a,29910.0,4.0,2015.0,0,0,0,0,aa,3642818,418588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1111,1645177,1587030,858869,145589,134552,71742,a,a,1900.0,6.0,2014.0,1,31,2013,"Jan,Apr,Jul,Oct",aa,4091076,351883
931,1112,3433630,2926635,1614029,275428,244299,133549,c,c,1880.0,4.0,2006.0,0,0,0,0,cc,7974294,653276
932,1113,1988864,2043271,1164107,221247,220065,120839,a,c,9260.0,0.0,0.0,0,0,0,0,ac,5196242,562151
933,1114,6165541,6227969,3809075,969963,973174,566405,a,c,870.0,0.0,0.0,0,0,0,0,ac,16202585,2509542


**Data Preprocessing: Promo2 Data**

In [12]:
promo2_df=store_data[store_data['Promo2']==1]



In [13]:
pre2013_promo2=promo2_df[promo2_df['Promo2SinceYear']<2013]
pre2013_promo2_list=promo2_df[promo2_df['Promo2SinceYear']<2013]['Store'].tolist()
from2013_promo2=promo2_df[promo2_df['Promo2SinceYear']==2013].reset_index()
from2013_promo2_list=promo2_df[promo2_df['Promo2SinceYear']==2013]['Store'].tolist()
from2014_promo2=promo2_df[promo2_df['Promo2SinceYear']==2014].reset_index()
from2014_promo2_list=promo2_df[promo2_df['Promo2SinceYear']==2014]['Store'].tolist()
from2015_promo2=promo2_df[promo2_df['Promo2SinceYear']==2015].reset_index()
from2015_promo2_list=promo2_df[promo2_df['Promo2SinceYear']==2015]['Store'].tolist()

**Stores that began Promo2 before 2013**

In [14]:
pre2013_promo2_int1=pre2013_promo2[pre2013_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']
pre2013_promo2_int1_list=pre2013_promo2[pre2013_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']['Store'].tolist()
pre2013_promo2_int2=pre2013_promo2[pre2013_promo2['PromoInterval']=='Feb,May,Aug,Nov']
pre2013_promo2_int2_list=pre2013_promo2[pre2013_promo2['PromoInterval']=='Feb,May,Aug,Nov']['Store'].tolist()
pre2013_promo2_int3=pre2013_promo2[pre2013_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']
pre2013_promo2_int3_list=pre2013_promo2[pre2013_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']['Store'].tolist()

In [15]:
open_days_y1=year1_data[year1_data['Open']==1]
open_days_y2=year2_data[year2_data['Open']==1]
open_days_y3=year3_data[year3_data['Open']==1]
open_days_store_y1=pd.DataFrame(open_days_y1.groupby(['Store','Month of Sale'])['Date'].count().reset_index())
open_days_store_y2=pd.DataFrame(open_days_y2.groupby(['Store','Month of Sale'])['Date'].count().reset_index())
open_days_store_y3=pd.DataFrame(open_days_y3.groupby(['Store','Month of Sale'])['Date'].count().reset_index())
open_days_store_y1_promo2_int1=open_days_store_y1[open_days_store_y1['Store'].isin(pre2013_promo2_int1_list)].reset_index()
open_days_store_y2_promo2_int1=open_days_store_y2[open_days_store_y2['Store'].isin(pre2013_promo2_int1_list)].reset_index()
open_days_store_y3_promo2_int1=open_days_store_y3[open_days_store_y3['Store'].isin(pre2013_promo2_int1_list)].reset_index()

open_days_store_y1_promo2_int2=open_days_store_y1[open_days_store_y1['Store'].isin(pre2013_promo2_int2_list)].reset_index()
open_days_store_y2_promo2_int2=open_days_store_y2[open_days_store_y2['Store'].isin(pre2013_promo2_int2_list)].reset_index()
open_days_store_y3_promo2_int2=open_days_store_y3[open_days_store_y3['Store'].isin(pre2013_promo2_int2_list)].reset_index()

open_days_store_y1_promo2_int3=open_days_store_y1[open_days_store_y1['Store'].isin(pre2013_promo2_int3_list)].reset_index()
open_days_store_y2_promo2_int3=open_days_store_y2[open_days_store_y2['Store'].isin(pre2013_promo2_int3_list)].reset_index()
open_days_store_y3_promo2_int3=open_days_store_y3[open_days_store_y3['Store'].isin(pre2013_promo2_int3_list)].reset_index()


In [16]:
noint1_list=[2,3,5,6,8,9,11,12]
noint2_list=[1,3,4,6,7,9,10,12]
noint3_list=[1,2,4,5,7,8,10,11]

In [17]:
open_days_store_y1_promo2_int1.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y2_promo2_int1.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_int1.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_int2.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y2_promo2_int2.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_int2.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_int3.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y2_promo2_int3.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_int3.loc[open_days_store_y1_promo2_int1['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_int1['Year of Sale']=([2013]*1824)
open_days_store_y2_promo2_int1['Year of Sale']=([2014]*1824)
open_days_store_y3_promo2_int1['Year of Sale']=([2015]*1064)

open_days_store_y1_promo2_int2['Year of Sale']=([2013]*600)
open_days_store_y2_promo2_int2['Year of Sale']=([2014]*600)
open_days_store_y3_promo2_int2['Year of Sale']=([2015]*350)

open_days_store_y1_promo2_int3['Year of Sale']=([2013]*672)
open_days_store_y2_promo2_int3['Year of Sale']=([2014]*672)
open_days_store_y3_promo2_int3['Year of Sale']=([2015]*392)


In [18]:
promo2_pre2013_final=open_days_store_y1_promo2_int1.append([open_days_store_y2_promo2_int1,open_days_store_y3_promo2_int1,open_days_store_y1_promo2_int2,open_days_store_y2_promo2_int2,open_days_store_y3_promo2_int2,open_days_store_y1_promo2_int3,open_days_store_y2_promo2_int3,open_days_store_y3_promo2_int3])

**Stores that began Promo2 In 2013**

In [19]:
from2013_promo2['Promo2SinceWeek-EG']=from2013_promo2['Promo2SinceWeek'].astype(str)
from2013_promo2['Promo2SinceYear-EG']=from2013_promo2['Promo2SinceYear'].astype(str)
from2013_promo2['Promo2SinceDate']=from2013_promo2['Promo2SinceYear-EG']+'-W'+from2013_promo2['Promo2SinceWeek-EG']

In [20]:
import datetime
Promo2_Since_Date=[]
for t in from2013_promo2['Promo2SinceDate'].tolist():
    eg=datetime.datetime.strptime(t+'-1','%Y-W%W-%w')
    Promo2_Since_Date.append(eg)
promo2_dates_final=pd.DataFrame(Promo2_Since_Date)
promo2_dates_final.columns=['Promo2Date']
promo2_dates_final['Promo2Month']=promo2_dates_final.Promo2Date.dt.month
from2013_promo2['Promo2Month']=promo2_dates_final['Promo2Month']
from2013_promo2_int1=from2013_promo2[from2013_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']
from2013_promo2_int1_list=from2013_promo2[from2013_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']['Store'].tolist()
from2013_promo2_int2=from2013_promo2[from2013_promo2['PromoInterval']=='Feb,May,Aug,Nov']
from2013_promo2_int2_list=from2013_promo2[from2013_promo2['PromoInterval']=='Feb,May,Aug,Nov']['Store'].tolist()
from2013_promo2_int3=from2013_promo2[from2013_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']
from2013_promo2_int3_list=from2013_promo2[from2013_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']['Store'].tolist()
from2013_promo2_int1.drop_duplicates(subset=['Promo2Month'])['Promo2Month'].tolist()


[4, 8, 1, 3, 10, 2, 7, 9]

**From 2013-Int1 -preprocessing**

In [21]:
from2013_promo2_int1_jan=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==1]
from2013_promo2_int1_jan_list=from2013_promo2_int1_jan['Store'].tolist()
from2013_promo2_int1_feb=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==2]
from2013_promo2_int1_feb_list=from2013_promo2_int1_feb['Store'].tolist()
from2013_promo2_int1_mar=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==3]
from2013_promo2_int1_mar_list=from2013_promo2_int1_mar['Store'].tolist()
from2013_promo2_int1_apr=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==4]
from2013_promo2_int1_apr_list=from2013_promo2_int1_apr['Store'].tolist()
from2013_promo2_int1_jul=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==7]
from2013_promo2_int1_jul_list=from2013_promo2_int1_jul['Store'].tolist()
from2013_promo2_int1_aug=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==8]
from2013_promo2_int1_aug_list=from2013_promo2_int1_aug['Store'].tolist()
from2013_promo2_int1_sept=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==9]
from2013_promo2_int1_sept_list=from2013_promo2_int1_sept['Store'].tolist()
from2013_promo2_int1_oct=from2013_promo2_int1[from2013_promo2_int1['Promo2Month']==10]
from2013_promo2_int1_oct_list=from2013_promo2_int1_oct['Store'].tolist()

In [22]:
open_days_store_y1_promo2_cat2_int1_jan=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_jan_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_jan=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_jan_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_jan=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_jan_list)].reset_index()

open_days_store_y1_promo2_cat2_int1_feb=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_feb_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_feb=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_feb_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_feb=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_feb_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_mar=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_mar_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_mar=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_mar_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_mar=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_mar_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_apr=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_apr_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_apr=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_apr_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_apr=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_apr_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_jul=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_jul_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_jul=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_jul_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_jul=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_jul_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_aug=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_aug_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_aug=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_aug_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_aug=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_aug_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_sept=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_sept_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_sept=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_sept_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_sept=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_sept_list)].reset_index()


open_days_store_y1_promo2_cat2_int1_oct=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int1_oct_list)].reset_index()
open_days_store_y2_promo2_cat2_int1_oct=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int1_oct_list)].reset_index()
open_days_store_y3_promo2_cat2_int1_oct=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int1_oct_list)].reset_index()


In [23]:
noint1_list
noint1_list_feb=noint1_list+[1]
noint1_list_jul=noint1_list_feb+[4]
noint1_list_aug=noint1_list_jul+[7]


In [24]:
open_days_store_y1_promo2_cat2_int1_jan.loc[open_days_store_y1_promo2_cat2_int1_jan['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y2_promo2_cat2_int1_jan.loc[open_days_store_y2_promo2_cat2_int1_jan['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_jan.loc[open_days_store_y3_promo2_cat2_int1_jan['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_cat2_int1_feb.loc[open_days_store_y1_promo2_cat2_int1_feb['Month of Sale'].isin(noint1_list_feb),'Date']=0
open_days_store_y2_promo2_cat2_int1_feb.loc[open_days_store_y2_promo2_cat2_int1_feb['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_feb.loc[open_days_store_y3_promo2_cat2_int1_feb['Month of Sale'].isin(noint1_list),'Date']=0


open_days_store_y1_promo2_cat2_int1_mar.loc[open_days_store_y1_promo2_cat2_int1_mar['Month of Sale'].isin(noint1_list_feb),'Date']=0
open_days_store_y2_promo2_cat2_int1_mar.loc[open_days_store_y2_promo2_cat2_int1_mar['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_mar.loc[open_days_store_y3_promo2_cat2_int1_mar['Month of Sale'].isin(noint1_list),'Date']=0


open_days_store_y1_promo2_cat2_int1_apr.loc[open_days_store_y1_promo2_cat2_int1_apr['Month of Sale'].isin(noint1_list_feb),'Date']=0
open_days_store_y2_promo2_cat2_int1_apr.loc[open_days_store_y1_promo2_cat2_int1_apr['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_apr.loc[open_days_store_y1_promo2_cat2_int1_apr['Month of Sale'].isin(noint1_list),'Date']=0


open_days_store_y1_promo2_cat2_int1_jul.loc[open_days_store_y1_promo2_cat2_int1_jul['Month of Sale'].isin(noint1_list_jul),'Date']=0
open_days_store_y2_promo2_cat2_int1_jul.loc[open_days_store_y2_promo2_cat2_int1_jul['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_jul.loc[open_days_store_y3_promo2_cat2_int1_jul['Month of Sale'].isin(noint1_list),'Date']=0


open_days_store_y1_promo2_cat2_int1_aug.loc[open_days_store_y1_promo2_cat2_int1_aug['Month of Sale'].isin(noint1_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int1_aug.loc[open_days_store_y2_promo2_cat2_int1_aug['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_aug.loc[open_days_store_y3_promo2_cat2_int1_aug['Month of Sale'].isin(noint1_list),'Date']=0


open_days_store_y1_promo2_cat2_int1_sept.loc[open_days_store_y1_promo2_cat2_int1_sept['Month of Sale'].isin(noint1_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int1_sept.loc[open_days_store_y2_promo2_cat2_int1_sept['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_sept.loc[open_days_store_y3_promo2_cat2_int1_sept['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_cat2_int1_oct.loc[open_days_store_y1_promo2_cat2_int1_oct['Month of Sale'].isin(noint1_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int1_oct.loc[open_days_store_y2_promo2_cat2_int1_oct['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat2_int1_oct.loc[open_days_store_y3_promo2_cat2_int1_oct['Month of Sale'].isin(noint1_list),'Date']=0

In [25]:
open_days_store_y1_promo2_cat2_int1_jan['Year of Sale']=([2013]*120)
open_days_store_y2_promo2_cat2_int1_jan['Year of Sale']=([2014]*120)
open_days_store_y3_promo2_cat2_int1_jan['Year of Sale']=([2015]*70)

open_days_store_y1_promo2_cat2_int1_feb['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat2_int1_feb['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat2_int1_feb['Year of Sale']=([2015]*14)

open_days_store_y1_promo2_cat2_int1_mar['Year of Sale']=([2013]*36)
open_days_store_y2_promo2_cat2_int1_mar['Year of Sale']=([2014]*36)
open_days_store_y3_promo2_cat2_int1_mar['Year of Sale']=([2015]*21)


open_days_store_y1_promo2_cat2_int1_apr['Year of Sale']=([2013]*48)
open_days_store_y2_promo2_cat2_int1_apr['Year of Sale']=([2014]*48)
open_days_store_y3_promo2_cat2_int1_apr['Year of Sale']=([2015]*28)


open_days_store_y1_promo2_cat2_int1_jul['Year of Sale']=([2013]*36)
open_days_store_y2_promo2_cat2_int1_jul['Year of Sale']=([2014]*36)
open_days_store_y3_promo2_cat2_int1_jul['Year of Sale']=([2015]*21)


open_days_store_y1_promo2_cat2_int1_aug['Year of Sale']=([2013]*156)
open_days_store_y2_promo2_cat2_int1_aug['Year of Sale']=([2014]*156)
open_days_store_y3_promo2_cat2_int1_aug['Year of Sale']=([2015]*91)

open_days_store_y1_promo2_cat2_int1_sept['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat2_int1_sept['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat2_int1_sept['Year of Sale']=([2015]*14)

open_days_store_y1_promo2_cat2_int1_oct['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat2_int1_oct['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat2_int1_oct['Year of Sale']=([2015]*14)

In [26]:
promo2_from2013_int1_final=open_days_store_y1_promo2_cat2_int1_jan.append([open_days_store_y2_promo2_cat2_int1_jan,open_days_store_y3_promo2_cat2_int1_jan,open_days_store_y1_promo2_cat2_int1_feb,open_days_store_y2_promo2_cat2_int1_feb,open_days_store_y3_promo2_cat2_int1_feb,open_days_store_y1_promo2_cat2_int1_mar,open_days_store_y2_promo2_cat2_int1_mar,open_days_store_y3_promo2_cat2_int1_feb,open_days_store_y1_promo2_cat2_int1_apr,open_days_store_y2_promo2_cat2_int1_apr,open_days_store_y3_promo2_cat2_int1_apr,open_days_store_y1_promo2_cat2_int1_jul,open_days_store_y2_promo2_cat2_int1_jul,open_days_store_y3_promo2_cat2_int1_jul,open_days_store_y1_promo2_cat2_int1_aug,open_days_store_y2_promo2_cat2_int1_aug,open_days_store_y3_promo2_cat2_int1_aug,open_days_store_y1_promo2_cat2_int1_sept,open_days_store_y2_promo2_cat2_int1_sept,open_days_store_y3_promo2_cat2_int1_sept,open_days_store_y1_promo2_cat2_int1_oct,open_days_store_y2_promo2_cat2_int1_oct,open_days_store_y3_promo2_cat2_int1_oct])

**From 2013-Int2 -preprocessing**

In [27]:
from2013_promo2_int2.groupby('Promo2Month')['Store'].count()
from2013_promo2_int2_feb=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==2]
from2013_promo2_int2_feb_list=from2013_promo2_int2_feb['Store'].tolist()
from2013_promo2_int2_mar=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==3]
from2013_promo2_int2_mar_list=from2013_promo2_int2_mar['Store'].tolist()
from2013_promo2_int2_apr=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==4]
from2013_promo2_int2_apr_list=from2013_promo2_int2_apr['Store'].tolist()
from2013_promo2_int2_may=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==5]
from2013_promo2_int2_may_list=from2013_promo2_int2_may['Store'].tolist()
from2013_promo2_int2_jul=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==7]
from2013_promo2_int2_jul_list=from2013_promo2_int2_jul['Store'].tolist()
from2013_promo2_int2_aug=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==8]
from2013_promo2_int2_aug_list=from2013_promo2_int2_aug['Store'].tolist()
from2013_promo2_int2_nov=from2013_promo2_int2[from2013_promo2_int2['Promo2Month']==11]
from2013_promo2_int2_nov_list=from2013_promo2_int2_nov['Store'].tolist()

open_days_store_y1_promo2_cat2_int2_feb=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_feb_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_feb=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_feb_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_feb=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_feb_list)].reset_index()

open_days_store_y1_promo2_cat2_int2_mar=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_mar_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_mar=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_mar_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_mar=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_mar_list)].reset_index()


open_days_store_y1_promo2_cat2_int2_apr=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_apr_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_apr=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_apr_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_apr=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_apr_list)].reset_index()


open_days_store_y1_promo2_cat2_int2_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_may_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_may_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_may_list)].reset_index()


open_days_store_y1_promo2_cat2_int2_jul=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_jul_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_jul=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_jul_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_jul=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_jul_list)].reset_index()


open_days_store_y1_promo2_cat2_int2_aug=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_aug_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_aug=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_aug_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_aug=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_aug_list)].reset_index()


open_days_store_y1_promo2_cat2_int2_nov=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int2_nov_list)].reset_index()
open_days_store_y2_promo2_cat2_int2_nov=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int2_nov_list)].reset_index()
open_days_store_y3_promo2_cat2_int2_nov=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int2_nov_list)].reset_index()

noint2_list_mar=noint2_list+[2]
noint2_list_jul=noint2_list_mar+[5]
noint2_list_aug=noint2_list_jul+[8]




In [28]:
open_days_store_y1_promo2_cat2_int2_feb.loc[open_days_store_y1_promo2_cat2_int2_feb['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y2_promo2_cat2_int2_feb.loc[open_days_store_y2_promo2_cat2_int2_feb['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_feb.loc[open_days_store_y3_promo2_cat2_int2_feb['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat2_int2_mar.loc[open_days_store_y1_promo2_cat2_int2_mar['Month of Sale'].isin(noint2_list_mar),'Date']=0
open_days_store_y2_promo2_cat2_int2_mar.loc[open_days_store_y2_promo2_cat2_int2_mar['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_mar.loc[open_days_store_y3_promo2_cat2_int2_mar['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat2_int2_apr.loc[open_days_store_y1_promo2_cat2_int2_apr['Month of Sale'].isin(noint2_list_mar),'Date']=0
open_days_store_y2_promo2_cat2_int2_apr.loc[open_days_store_y2_promo2_cat2_int2_apr['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_apr.loc[open_days_store_y3_promo2_cat2_int2_apr['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat2_int2_may.loc[open_days_store_y1_promo2_cat2_int2_may['Month of Sale'].isin(noint2_list_mar),'Date']=0
open_days_store_y2_promo2_cat2_int2_may.loc[open_days_store_y2_promo2_cat2_int2_may['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_may.loc[open_days_store_y3_promo2_cat2_int2_may['Month of Sale'].isin(noint2_list),'Date']=0


open_days_store_y1_promo2_cat2_int2_jul.loc[open_days_store_y1_promo2_cat2_int2_jul['Month of Sale'].isin(noint2_list_jul),'Date']=0
open_days_store_y2_promo2_cat2_int2_jul.loc[open_days_store_y2_promo2_cat2_int2_jul['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_jul.loc[open_days_store_y3_promo2_cat2_int2_jul['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat2_int2_aug.loc[open_days_store_y1_promo2_cat2_int2_aug['Month of Sale'].isin(noint2_list_jul),'Date']=0
open_days_store_y2_promo2_cat2_int2_aug.loc[open_days_store_y2_promo2_cat2_int2_aug['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_aug.loc[open_days_store_y3_promo2_cat2_int2_aug['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat2_int2_nov.loc[open_days_store_y1_promo2_cat2_int2_nov['Month of Sale'].isin(noint2_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int2_nov.loc[open_days_store_y2_promo2_cat2_int2_nov['Month of Sale'].isin(noint2_list),'Date']=0
open_days_store_y3_promo2_cat2_int2_nov.loc[open_days_store_y3_promo2_cat2_int2_nov['Month of Sale'].isin(noint2_list),'Date']=0


In [29]:
open_days_store_y1_promo2_cat2_int2_feb['Year of Sale']=([2013]*324)
open_days_store_y2_promo2_cat2_int2_feb['Year of Sale']=([2014]*324)
open_days_store_y3_promo2_cat2_int2_feb['Year of Sale']=([2015]*189)

open_days_store_y1_promo2_cat2_int2_mar['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int2_mar['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int2_mar['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int2_apr['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat2_int2_apr['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat2_int2_apr['Year of Sale']=([2015]*14)

open_days_store_y1_promo2_cat2_int2_may['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int2_may['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int2_may['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int2_jul['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int2_jul['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int2_jul['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int2_aug['Year of Sale']=([2013]*216)
open_days_store_y2_promo2_cat2_int2_aug['Year of Sale']=([2014]*216)
open_days_store_y3_promo2_cat2_int2_aug['Year of Sale']=([2015]*126)

open_days_store_y1_promo2_cat2_int2_nov['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int2_nov['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int2_nov['Year of Sale']=([2015]*7)

In [30]:
promo2_from2013_int2_final=open_days_store_y1_promo2_cat2_int2_feb.append([open_days_store_y2_promo2_cat2_int2_feb,open_days_store_y3_promo2_cat2_int2_feb,open_days_store_y1_promo2_cat2_int2_mar,open_days_store_y2_promo2_cat2_int2_mar,open_days_store_y3_promo2_cat2_int2_mar,open_days_store_y1_promo2_cat2_int2_apr,open_days_store_y2_promo2_cat2_int2_apr,open_days_store_y3_promo2_cat2_int2_apr,open_days_store_y1_promo2_cat2_int2_may,open_days_store_y2_promo2_cat2_int2_may,open_days_store_y3_promo2_cat2_int2_may,open_days_store_y1_promo2_cat2_int2_jul,open_days_store_y2_promo2_cat2_int2_jul,open_days_store_y3_promo2_cat2_int2_jul,open_days_store_y1_promo2_cat2_int2_aug,open_days_store_y2_promo2_cat2_int2_aug,open_days_store_y3_promo2_cat2_int2_aug,open_days_store_y1_promo2_cat2_int2_nov,open_days_store_y2_promo2_cat2_int2_nov,open_days_store_y3_promo2_cat2_int2_nov])

**From 2013-Int3 Preprocessing**

In [31]:
from2013_promo2_int3.groupby('Promo2Month')['Store'].count()
from2013_promo2_int3_mar=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==3]
from2013_promo2_int3_mar_list=from2013_promo2_int3_mar['Store'].tolist()
from2013_promo2_int3_apr=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==4]
from2013_promo2_int3_apr_list=from2013_promo2_int3_apr['Store'].tolist()
from2013_promo2_int3_may=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==5]
from2013_promo2_int3_may_list=from2013_promo2_int3_may['Store'].tolist()
from2013_promo2_int3_jun=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==6]
from2013_promo2_int3_jun_list=from2013_promo2_int3_jun['Store'].tolist()
from2013_promo2_int3_aug=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==8]
from2013_promo2_int3_aug_list=from2013_promo2_int3_aug['Store'].tolist()
from2013_promo2_int3_sept=from2013_promo2_int3[from2013_promo2_int3['Promo2Month']==9]
from2013_promo2_int3_sept_list=from2013_promo2_int3_sept['Store'].tolist()

open_days_store_y1_promo2_cat2_int3_mar=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_mar_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_mar=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_mar_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_mar=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_mar_list)].reset_index()


open_days_store_y1_promo2_cat2_int3_apr=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_apr_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_apr=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_apr_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_apr=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_apr_list)].reset_index()


open_days_store_y1_promo2_cat2_int3_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_may_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_may_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_may_list)].reset_index()


open_days_store_y1_promo2_cat2_int3_jun=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_jun_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_jun=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_jun_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_jun=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_jun_list)].reset_index()


open_days_store_y1_promo2_cat2_int3_aug=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_aug_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_aug=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_aug_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_aug=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_aug_list)].reset_index()


open_days_store_y1_promo2_cat2_int3_sept=open_days_store_y1[open_days_store_y1['Store'].isin(from2013_promo2_int3_sept_list)].reset_index()
open_days_store_y2_promo2_cat2_int3_sept=open_days_store_y2[open_days_store_y2['Store'].isin(from2013_promo2_int3_sept_list)].reset_index()
open_days_store_y3_promo2_cat2_int3_sept=open_days_store_y3[open_days_store_y3['Store'].isin(from2013_promo2_int3_sept_list)].reset_index()



noint3_list_apr=noint3_list+[3]
noint3_list_aug=noint3_list_apr+[6]

open_days_store_y1_promo2_cat2_int3_mar.loc[open_days_store_y1_promo2_cat2_int3_mar['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y2_promo2_cat2_int3_mar.loc[open_days_store_y2_promo2_cat2_int3_mar['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_mar.loc[open_days_store_y3_promo2_cat2_int3_mar['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_apr.loc[open_days_store_y1_promo2_cat2_int3_apr['Month of Sale'].isin(noint3_list_apr),'Date']=0
open_days_store_y2_promo2_cat2_int3_apr.loc[open_days_store_y2_promo2_cat2_int3_apr['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_apr.loc[open_days_store_y3_promo2_cat2_int3_apr['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_may.loc[open_days_store_y1_promo2_cat2_int3_may['Month of Sale'].isin(noint3_list_apr),'Date']=0
open_days_store_y2_promo2_cat2_int3_may.loc[open_days_store_y2_promo2_cat2_int3_may['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_may.loc[open_days_store_y3_promo2_cat2_int3_may['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_jun.loc[open_days_store_y1_promo2_cat2_int3_jun['Month of Sale'].isin(noint3_list_apr),'Date']=0
open_days_store_y2_promo2_cat2_int3_jun.loc[open_days_store_y2_promo2_cat2_int3_jun['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_jun.loc[open_days_store_y3_promo2_cat2_int3_jun['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_aug.loc[open_days_store_y1_promo2_cat2_int3_aug['Month of Sale'].isin(noint3_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int3_aug.loc[open_days_store_y2_promo2_cat2_int3_aug['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_aug.loc[open_days_store_y3_promo2_cat2_int3_aug['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_sept.loc[open_days_store_y1_promo2_cat2_int3_sept['Month of Sale'].isin(noint3_list_aug),'Date']=0
open_days_store_y2_promo2_cat2_int3_sept.loc[open_days_store_y2_promo2_cat2_int3_sept['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat2_int3_sept.loc[open_days_store_y3_promo2_cat2_int3_sept['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat2_int3_mar['Year of Sale']=([2013]*60)
open_days_store_y2_promo2_cat2_int3_mar['Year of Sale']=([2014]*60)
open_days_store_y3_promo2_cat2_int3_mar['Year of Sale']=([2015]*35)

open_days_store_y1_promo2_cat2_int3_apr['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int3_apr['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int3_apr['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int3_may['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int3_may['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int3_may['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int3_jun['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int3_jun['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int3_jun['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int3_aug['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat2_int3_aug['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat2_int3_aug['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat2_int3_sept['Year of Sale']=([2013]*84)
open_days_store_y2_promo2_cat2_int3_sept['Year of Sale']=([2014]*84)
open_days_store_y3_promo2_cat2_int3_sept['Year of Sale']=([2015]*49)

promo2_from2013_int3_final=open_days_store_y1_promo2_cat2_int3_mar.append([open_days_store_y2_promo2_cat2_int3_mar,open_days_store_y3_promo2_cat2_int3_mar,open_days_store_y1_promo2_cat2_int3_apr,open_days_store_y2_promo2_cat2_int3_apr,open_days_store_y3_promo2_cat2_int3_apr,open_days_store_y1_promo2_cat2_int3_may,open_days_store_y2_promo2_cat2_int3_may,open_days_store_y3_promo2_cat2_int3_may,open_days_store_y1_promo2_cat2_int3_jun,open_days_store_y2_promo2_cat2_int3_jun,open_days_store_y3_promo2_cat2_int3_jun,open_days_store_y1_promo2_cat2_int3_aug,open_days_store_y2_promo2_cat2_int3_aug,open_days_store_y3_promo2_cat2_int3_aug,open_days_store_y1_promo2_cat2_int3_sept,open_days_store_y2_promo2_cat2_int3_sept,open_days_store_y3_promo2_cat2_int3_sept])

**Stores that began Promo2 in 2014**

In [34]:
from2014_promo2['Promo2SinceWeek-EG']=from2014_promo2['Promo2SinceWeek'].astype(str)
from2014_promo2['Promo2SinceYear-EG']=from2014_promo2['Promo2SinceYear'].astype(str)
from2014_promo2['Promo2SinceDate']=from2014_promo2['Promo2SinceYear-EG']+'-W'+from2014_promo2['Promo2SinceWeek-EG']


In [35]:
Promo2_Since_Date=[]
for t in from2014_promo2['Promo2SinceDate'].tolist():
    eg=datetime.datetime.strptime(t+'-1','%Y-W%W-%w')
    Promo2_Since_Date.append(eg)
promo2_dates_final=pd.DataFrame(Promo2_Since_Date)
promo2_dates_final.columns=['Promo2Date']
promo2_dates_final['Promo2Month']=promo2_dates_final.Promo2Date.dt.month
from2014_promo2['Promo2Month']=promo2_dates_final['Promo2Month']


In [36]:
from2014_promo2_int1=from2014_promo2[from2014_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']
from2014_promo2_int1_list=from2014_promo2[from2014_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']['Store'].tolist()
from2014_promo2_int2=from2014_promo2[from2014_promo2['PromoInterval']=='Feb,May,Aug,Nov']
from2014_promo2_int2_list=from2014_promo2[from2014_promo2['PromoInterval']=='Feb,May,Aug,Nov']['Store'].tolist()
from2014_promo2_int3=from2014_promo2[from2014_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']
from2014_promo2_int3_list=from2014_promo2[from2014_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']['Store'].tolist()

**From 2014-Int1 Preprocessing**

In [37]:
from2014_promo2_int1_jan=from2014_promo2_int1[from2014_promo2_int1['Promo2Month']==1]
from2014_promo2_int1_jan_list=from2014_promo2_int1_jan['Store'].tolist()

from2014_promo2_int1_mar=from2014_promo2_int1[from2014_promo2_int1['Promo2Month']==3]
from2014_promo2_int1_mar_list=from2014_promo2_int1_mar['Store'].tolist()

from2014_promo2_int1_apr=from2014_promo2_int1[from2014_promo2_int1['Promo2Month']==4]
from2014_promo2_int1_apr_list=from2014_promo2_int1_apr['Store'].tolist()

from2014_promo2_int1_may=from2014_promo2_int1[from2014_promo2_int1['Promo2Month']==5]
from2014_promo2_int1_may_list=from2014_promo2_int1_apr['Store'].tolist()


open_days_store_y1_promo2_cat3_int1_jan=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int1_jan_list)].reset_index()
open_days_store_y2_promo2_cat3_int1_jan=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int1_jan_list)].reset_index()
open_days_store_y3_promo2_cat3_int1_jan=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int1_jan_list)].reset_index()

open_days_store_y1_promo2_cat3_int1_mar=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int1_mar_list)].reset_index()
open_days_store_y2_promo2_cat3_int1_mar=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int1_mar_list)].reset_index()
open_days_store_y3_promo2_cat3_int1_mar=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int1_mar_list)].reset_index()

open_days_store_y1_promo2_cat3_int1_apr=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int1_apr_list)].reset_index()
open_days_store_y2_promo2_cat3_int1_apr=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int1_apr_list)].reset_index()
open_days_store_y3_promo2_cat3_int1_apr=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int1_apr_list)].reset_index()

open_days_store_y1_promo2_cat3_int1_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int1_may_list)].reset_index()
open_days_store_y2_promo2_cat3_int1_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int1_may_list)].reset_index()
open_days_store_y3_promo2_cat3_int1_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int1_may_list)].reset_index()


noint1_list_feb=noint1_list+[1]
noint1_list_may=noint1_list_feb+[4]
noint1_list_aug=noint1_list_may+[7]

all_months=[1,2,3,4,5,6,7,8,9,10,11,12]

In [38]:
open_days_store_y1_promo2_cat3_int1_jan.loc[open_days_store_y1_promo2_cat3_int1_jan['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int1_jan.loc[open_days_store_y2_promo2_cat3_int1_jan['Month of Sale'].isin(noint1_list),'Date']=0
open_days_store_y3_promo2_cat3_int1_jan.loc[open_days_store_y3_promo2_cat3_int1_jan['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_cat3_int1_mar.loc[open_days_store_y1_promo2_cat3_int1_mar['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int1_mar.loc[open_days_store_y2_promo2_cat3_int1_mar['Month of Sale'].isin(noint1_list_feb),'Date']=0
open_days_store_y3_promo2_cat3_int1_mar.loc[open_days_store_y3_promo2_cat3_int1_mar['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_cat3_int1_apr.loc[open_days_store_y1_promo2_cat3_int1_apr['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int1_apr.loc[open_days_store_y2_promo2_cat3_int1_apr['Month of Sale'].isin(noint1_list_feb),'Date']=0
open_days_store_y3_promo2_cat3_int1_apr.loc[open_days_store_y3_promo2_cat3_int1_apr['Month of Sale'].isin(noint1_list),'Date']=0

open_days_store_y1_promo2_cat3_int1_may.loc[open_days_store_y1_promo2_cat3_int1_may['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int1_may.loc[open_days_store_y2_promo2_cat3_int1_may['Month of Sale'].isin(noint1_list_may),'Date']=0
open_days_store_y3_promo2_cat3_int1_may.loc[open_days_store_y3_promo2_cat3_int1_may['Month of Sale'].isin(noint1_list),'Date']=0



In [39]:
open_days_store_y1_promo2_cat3_int1_jan['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat3_int1_jan['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat3_int1_jan['Year of Sale']=([2015]*14)

open_days_store_y1_promo2_cat3_int1_mar['Year of Sale']=([2013]*156)
open_days_store_y2_promo2_cat3_int1_mar['Year of Sale']=([2014]*156)
open_days_store_y3_promo2_cat3_int1_mar['Year of Sale']=([2015]*91)

open_days_store_y1_promo2_cat3_int1_apr['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat3_int1_apr['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat3_int1_apr['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat3_int1_may['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat3_int1_may['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat3_int1_may['Year of Sale']=([2015]*7)

promo2_from2014_int1_final=open_days_store_y1_promo2_cat3_int1_jan.append([open_days_store_y2_promo2_cat3_int1_jan,open_days_store_y3_promo2_cat3_int1_jan,open_days_store_y1_promo2_cat3_int1_mar,open_days_store_y2_promo2_cat3_int1_mar,open_days_store_y3_promo2_cat3_int1_mar,open_days_store_y1_promo2_cat3_int1_apr,open_days_store_y2_promo2_cat3_int1_apr,open_days_store_y3_promo2_cat3_int1_apr,open_days_store_y1_promo2_cat3_int1_may,open_days_store_y2_promo2_cat3_int1_may,open_days_store_y3_promo2_cat3_int1_may])

**From 2014-Int2 Preprocessing**

In [41]:
from2014_promo2_int2.groupby('Promo2Month')['Store'].count()
from2014_promo2_int2_may=from2014_promo2_int2[from2014_promo2_int2['Promo2Month']==5]
from2014_promo2_int2_may_list=from2014_promo2_int2_may['Store'].tolist()

from2014_promo2_int2_nov=from2014_promo2_int2[from2014_promo2_int2['Promo2Month']==11]
from2014_promo2_int2_nov_list=from2014_promo2_int2_nov['Store'].tolist()



In [42]:
open_days_store_y1_promo2_cat3_int2_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int2_may_list)].reset_index()
open_days_store_y2_promo2_cat3_int2_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int2_may_list)].reset_index()
open_days_store_y3_promo2_cat3_int2_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int2_may_list)].reset_index()

open_days_store_y1_promo2_cat3_int2_nov=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int2_nov_list)].reset_index()
open_days_store_y2_promo2_cat3_int2_nov=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int2_nov_list)].reset_index()
open_days_store_y3_promo2_cat3_int2_nov=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int2_nov_list)].reset_index()


In [43]:
open_days_store_y1_promo2_cat3_int2_may.loc[open_days_store_y1_promo2_cat3_int2_may['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int2_may.loc[open_days_store_y2_promo2_cat3_int2_may['Month of Sale'].isin(noint2_list_mar),'Date']=0
open_days_store_y3_promo2_cat3_int2_may.loc[open_days_store_y3_promo2_cat3_int2_may['Month of Sale'].isin(noint2_list),'Date']=0

open_days_store_y1_promo2_cat3_int2_nov.loc[open_days_store_y1_promo2_cat3_int2_nov['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int2_nov.loc[open_days_store_y2_promo2_cat3_int2_nov['Month of Sale'].isin(noint2_list_aug),'Date']=0
open_days_store_y3_promo2_cat3_int2_nov.loc[open_days_store_y3_promo2_cat3_int2_nov['Month of Sale'].isin(noint2_list),'Date']=0


In [44]:
open_days_store_y1_promo2_cat3_int2_may['Year of Sale']=([2013]*36)
open_days_store_y2_promo2_cat3_int2_may['Year of Sale']=([2014]*36)
open_days_store_y3_promo2_cat3_int2_may['Year of Sale']=([2015]*21)

open_days_store_y1_promo2_cat3_int2_nov['Year of Sale']=([2013]*36)
open_days_store_y2_promo2_cat3_int2_nov['Year of Sale']=([2014]*36)
open_days_store_y3_promo2_cat3_int2_nov['Year of Sale']=([2015]*21)

promo2_from2014_int2_final=open_days_store_y1_promo2_cat3_int2_may.append([open_days_store_y2_promo2_cat3_int2_may,open_days_store_y3_promo2_cat3_int2_may,open_days_store_y1_promo2_cat3_int2_nov,open_days_store_y2_promo2_cat3_int2_nov,open_days_store_y3_promo2_cat3_int2_nov])

**From 2014-Int3-Preprocessing**

In [45]:
from2014_promo2_int3_jan=from2014_promo2_int3[from2014_promo2_int3['Promo2Month']==1]
from2014_promo2_int3_jan_list=from2014_promo2_int3_jan['Store'].tolist()

from2014_promo2_int3_mar=from2014_promo2_int3[from2014_promo2_int3['Promo2Month']==3]
from2014_promo2_int3_mar_list=from2014_promo2_int3_mar['Store'].tolist()

from2014_promo2_int3_may=from2014_promo2_int3[from2014_promo2_int3['Promo2Month']==5]
from2014_promo2_int3_may_list=from2014_promo2_int3_may['Store'].tolist()

from2014_promo2_int3_jun=from2014_promo2_int3[from2014_promo2_int3['Promo2Month']==6]
from2014_promo2_int3_jun_list=from2014_promo2_int3_jun['Store'].tolist()


open_days_store_y1_promo2_cat3_int3_jan=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int3_jan_list)].reset_index()
open_days_store_y2_promo2_cat3_int3_jan=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int3_jan_list)].reset_index()
open_days_store_y3_promo2_cat3_int3_jan=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int3_jan_list)].reset_index()

open_days_store_y1_promo2_cat3_int3_mar=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int3_mar_list)].reset_index()
open_days_store_y2_promo2_cat3_int3_mar=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int3_mar_list)].reset_index()
open_days_store_y3_promo2_cat3_int3_mar=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int3_mar_list)].reset_index()

open_days_store_y1_promo2_cat3_int3_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int3_may_list)].reset_index()
open_days_store_y2_promo2_cat3_int3_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int3_may_list)].reset_index()
open_days_store_y3_promo2_cat3_int3_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int3_may_list)].reset_index()

open_days_store_y1_promo2_cat3_int3_jun=open_days_store_y1[open_days_store_y1['Store'].isin(from2014_promo2_int3_jun_list)].reset_index()
open_days_store_y2_promo2_cat3_int3_jun=open_days_store_y2[open_days_store_y2['Store'].isin(from2014_promo2_int3_jun_list)].reset_index()
open_days_store_y3_promo2_cat3_int3_jun=open_days_store_y3[open_days_store_y3['Store'].isin(from2014_promo2_int3_jun_list)].reset_index()


open_days_store_y1_promo2_cat3_int3_jan.loc[open_days_store_y1_promo2_cat3_int3_jan['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int3_jan.loc[open_days_store_y2_promo2_cat3_int3_jan['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat3_int3_jan.loc[open_days_store_y3_promo2_cat3_int3_jan['Month of Sale'].isin(noint3_list),'Date']=0


open_days_store_y1_promo2_cat3_int3_mar.loc[open_days_store_y1_promo2_cat3_int3_mar['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int3_mar.loc[open_days_store_y2_promo2_cat3_int3_mar['Month of Sale'].isin(noint3_list),'Date']=0
open_days_store_y3_promo2_cat3_int3_mar.loc[open_days_store_y3_promo2_cat3_int3_mar['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat3_int3_may.loc[open_days_store_y1_promo2_cat3_int3_may['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int3_may.loc[open_days_store_y2_promo2_cat3_int3_may['Month of Sale'].isin(noint3_list_apr),'Date']=0
open_days_store_y3_promo2_cat3_int3_may.loc[open_days_store_y3_promo2_cat3_int3_may['Month of Sale'].isin(noint3_list),'Date']=0


open_days_store_y1_promo2_cat3_int3_jun.loc[open_days_store_y1_promo2_cat3_int3_jun['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat3_int3_jun.loc[open_days_store_y2_promo2_cat3_int3_jun['Month of Sale'].isin(noint3_list_apr),'Date']=0
open_days_store_y3_promo2_cat3_int3_jun.loc[open_days_store_y3_promo2_cat3_int3_jun['Month of Sale'].isin(noint3_list),'Date']=0




In [46]:
open_days_store_y1_promo2_cat3_int3_jan['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat3_int3_jan['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat3_int3_jan['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat3_int3_mar['Year of Sale']=([2013]*192)
open_days_store_y2_promo2_cat3_int3_mar['Year of Sale']=([2014]*192)
open_days_store_y3_promo2_cat3_int3_mar['Year of Sale']=([2015]*112)

open_days_store_y1_promo2_cat3_int3_may['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat3_int3_may['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat3_int3_may['Year of Sale']=([2015]*7)


open_days_store_y1_promo2_cat3_int3_jun['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat3_int3_jun['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat3_int3_jun['Year of Sale']=([2015]*7)



promo2_from2014_int3_final=open_days_store_y1_promo2_cat3_int3_jan.append([open_days_store_y2_promo2_cat3_int3_jan,open_days_store_y3_promo2_cat3_int3_jan,open_days_store_y1_promo2_cat3_int3_mar,open_days_store_y2_promo2_cat3_int3_mar,open_days_store_y3_promo2_cat3_int3_mar,open_days_store_y1_promo2_cat3_int3_may,open_days_store_y2_promo2_cat3_int3_may,open_days_store_y3_promo2_cat3_int3_may,open_days_store_y1_promo2_cat3_int3_jun,open_days_store_y2_promo2_cat3_int3_jun,open_days_store_y3_promo2_cat3_int3_jun])

**Stores that started Promo2 in 2015**

In [47]:
from2015_promo2['Promo2SinceWeek-EG']=from2015_promo2['Promo2SinceWeek'].astype(str)
from2015_promo2['Promo2SinceYear-EG']=from2015_promo2['Promo2SinceYear'].astype(str)
from2015_promo2['Promo2SinceDate']=from2015_promo2['Promo2SinceYear-EG']+'-W'+from2015_promo2['Promo2SinceWeek-EG']

Promo2_Since_Date=[]
for t in from2015_promo2['Promo2SinceDate'].tolist():
    eg=datetime.datetime.strptime(t+'-1','%Y-W%W-%w')
    Promo2_Since_Date.append(eg)
promo2_dates_final=pd.DataFrame(Promo2_Since_Date)
promo2_dates_final.columns=['Promo2Date']
promo2_dates_final['Promo2Month']=promo2_dates_final.Promo2Date.dt.month
from2015_promo2['Promo2Month']=promo2_dates_final['Promo2Month']

from2015_promo2_int1=from2015_promo2[from2015_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']
from2015_promo2_int1_list=from2015_promo2[from2015_promo2['PromoInterval']=='Jan,Apr,Jul,Oct']['Store'].tolist()
from2015_promo2_int2=from2015_promo2[from2015_promo2['PromoInterval']=='Feb,May,Aug,Nov']
from2015_promo2_int2_list=from2015_promo2[from2015_promo2['PromoInterval']=='Feb,May,Aug,Nov']['Store'].tolist()
from2015_promo2_int3=from2015_promo2[from2015_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']
from2015_promo2_int3_list=from2015_promo2[from2015_promo2['PromoInterval']=='Mar,Jun,Sept,Dec']['Store'].tolist()

**From 2015-Int 1 Preprocessing**

In [49]:
from2015_promo2_int1_apr=from2015_promo2_int1[from2015_promo2_int1['Promo2Month']==4]
from2015_promo2_int1_apr_list=from2015_promo2_int1_apr['Store'].tolist()
open_days_store_y1_promo2_cat4_int1_apr=open_days_store_y1[open_days_store_y1['Store'].isin(from2015_promo2_int1_apr_list)].reset_index()
open_days_store_y2_promo2_cat4_int1_apr=open_days_store_y2[open_days_store_y2['Store'].isin(from2015_promo2_int1_apr_list)].reset_index()
open_days_store_y3_promo2_cat4_int1_apr=open_days_store_y3[open_days_store_y3['Store'].isin(from2015_promo2_int1_apr_list)].reset_index()
open_days_store_y1_promo2_cat4_int1_apr.loc[open_days_store_y1_promo2_cat4_int1_apr['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat4_int1_apr.loc[open_days_store_y1_promo2_cat4_int1_apr['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y3_promo2_cat4_int1_apr.loc[open_days_store_y1_promo2_cat4_int1_apr['Month of Sale'].isin(noint1_list_feb),'Date']=0

open_days_store_y1_promo2_cat4_int1_apr['Year of Sale']=([2013]*48)
open_days_store_y2_promo2_cat4_int1_apr['Year of Sale']=([2014]*48)
open_days_store_y3_promo2_cat4_int1_apr['Year of Sale']=([2015]*28)

promo2_from2015_int1_final=open_days_store_y1_promo2_cat4_int1_apr.append([open_days_store_y2_promo2_cat4_int1_apr,open_days_store_y3_promo2_cat4_int1_apr])

**From 2015-Int 2 Preprocessing**

In [50]:
from2015_promo2_int2_may=from2015_promo2_int2[from2015_promo2_int2['Promo2Month']==5]
from2015_promo2_int2_may_list=from2015_promo2_int2_may['Store'].tolist()
open_days_store_y1_promo2_cat4_int2_may=open_days_store_y1[open_days_store_y1['Store'].isin(from2015_promo2_int2_may_list)].reset_index()
open_days_store_y2_promo2_cat4_int2_may=open_days_store_y2[open_days_store_y2['Store'].isin(from2015_promo2_int2_may_list)].reset_index()
open_days_store_y3_promo2_cat4_int2_may=open_days_store_y3[open_days_store_y3['Store'].isin(from2015_promo2_int2_may_list)].reset_index()
open_days_store_y1_promo2_cat4_int2_may.loc[open_days_store_y1_promo2_cat4_int2_may['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat4_int2_may.loc[open_days_store_y2_promo2_cat4_int2_may['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y3_promo2_cat4_int2_may.loc[open_days_store_y3_promo2_cat4_int2_may['Month of Sale'].isin(noint2_list_mar),'Date']=0

open_days_store_y1_promo2_cat4_int2_may['Year of Sale']=([2013]*24)
open_days_store_y2_promo2_cat4_int2_may['Year of Sale']=([2014]*24)
open_days_store_y3_promo2_cat4_int2_may['Year of Sale']=([2015]*14)

promo2_from2015_int2_final=open_days_store_y1_promo2_cat4_int2_may.append([open_days_store_y2_promo2_cat4_int2_may,open_days_store_y3_promo2_cat4_int2_may])

**From 2015-Int 3 Preprocessing**

In [51]:
from2015_promo2_int3_feb=from2015_promo2_int3[from2015_promo2_int3['Promo2Month']==2]
from2015_promo2_int3_feb_list=from2015_promo2_int3_feb['Store'].tolist()

from2015_promo2_int3_jun=from2015_promo2_int3[from2015_promo2_int3['Promo2Month']==6]
from2015_promo2_int3_jun_list=from2015_promo2_int3_jun['Store'].tolist()

open_days_store_y1_promo2_cat4_int3_feb=open_days_store_y1[open_days_store_y1['Store'].isin(from2015_promo2_int3_feb_list)].reset_index()
open_days_store_y2_promo2_cat4_int3_feb=open_days_store_y2[open_days_store_y2['Store'].isin(from2015_promo2_int3_feb_list)].reset_index()
open_days_store_y3_promo2_cat4_int3_feb=open_days_store_y3[open_days_store_y3['Store'].isin(from2015_promo2_int3_feb_list)].reset_index()

open_days_store_y1_promo2_cat4_int3_jun=open_days_store_y1[open_days_store_y1['Store'].isin(from2015_promo2_int3_jun_list)].reset_index()
open_days_store_y2_promo2_cat4_int3_jun=open_days_store_y2[open_days_store_y2['Store'].isin(from2015_promo2_int3_jun_list)].reset_index()
open_days_store_y3_promo2_cat4_int3_jun=open_days_store_y3[open_days_store_y3['Store'].isin(from2015_promo2_int3_jun_list)].reset_index()


open_days_store_y1_promo2_cat4_int3_feb.loc[open_days_store_y1_promo2_cat4_int3_feb['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat4_int3_feb.loc[open_days_store_y2_promo2_cat4_int3_feb['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y3_promo2_cat4_int3_feb.loc[open_days_store_y3_promo2_cat4_int3_feb['Month of Sale'].isin(noint3_list),'Date']=0

open_days_store_y1_promo2_cat4_int3_jun.loc[open_days_store_y1_promo2_cat4_int3_jun['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y2_promo2_cat4_int3_jun.loc[open_days_store_y2_promo2_cat4_int3_jun['Month of Sale'].isin(all_months),'Date']=0
open_days_store_y3_promo2_cat4_int3_jun.loc[open_days_store_y3_promo2_cat4_int3_jun['Month of Sale'].isin(noint3_list_apr),'Date']=0

open_days_store_y1_promo2_cat4_int3_feb['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat4_int3_feb['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat4_int3_feb['Year of Sale']=([2015]*7)

open_days_store_y1_promo2_cat4_int3_jun['Year of Sale']=([2013]*12)
open_days_store_y2_promo2_cat4_int3_jun['Year of Sale']=([2014]*12)
open_days_store_y3_promo2_cat4_int3_jun['Year of Sale']=([2015]*7)

promo2_from2015_int3_final=open_days_store_y1_promo2_cat4_int3_feb.append([open_days_store_y2_promo2_cat4_int3_feb,open_days_store_y3_promo2_cat4_int3_feb,open_days_store_y1_promo2_cat4_int3_jun,open_days_store_y2_promo2_cat4_int3_jun,open_days_store_y3_promo2_cat4_int3_jun])
final_promo2=promo2_pre2013_final.append([promo2_from2013_int1_final,promo2_from2013_int2_final,promo2_from2013_int3_final,promo2_from2014_int1_final,promo2_from2014_int2_final,promo2_from2014_int3_final,promo2_from2015_int1_final,promo2_from2015_int2_final,promo2_from2015_int3_final])

In [52]:
final_promo2

,index,Store,Month of Sale,Date,Year of Sale
0,12,2,1,26,2013
1,13,2,2,0,2013
2,14,2,3,0,2013
3,15,2,4,25,2013
4,16,2,5,0,2013
...,...,...,...,...,...
2,5098,872,3,0,2015
3,5099,872,4,0,2015
4,5100,872,5,0,2015
5,5101,872,6,25,2015
